In [ ]:
# !pip install torch==1.8
# !pip uninstall torchvision
# !pip install torchvision

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import time
from torch.autograd import Variable, Function
from torchsummary import summary
from torchvision import transforms, datasets
from torchvision.datasets import VisionDataset, ImageFolder
import matplotlib.pyplot as plt
import itertools
import math
from torch.nn.modules.utils import _pair

In [ ]:
# !pip install torch==1.6
# !pip install torchvision==0.8.1

In [ ]:
BATCH_SIZE = 100
NUM_CLASSES = 10

# Dataset

In [ ]:
transform = transforms.Compose([
  # transforms.RandomResizedCrop(224),
  transforms.ToTensor()
])

In [ ]:
!rm -rf caltech101/

In [ ]:
# torchvision.datasets.

In [ ]:
caltech_data = datasets.ImageFolder("./caltech101/101_ObjectCategories", transform=transform)
caltech_data_size = len(caltech_data)
train_data_size = int(0.7 * caltech_data_size)
val_data_size = (caltech_data_size - train_data_size)//2
test_data_size = caltech_data_size - train_data_size - val_data_size

In [ ]:
train_data, val_test_data = torch.utils.data.random_split(caltech_data, [train_data_size, caltech_data_size-train_data_size])
val_data, test_data = torch.utils.data.random_split(val_test_data, [val_data_size, test_data_size])

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2, drop_last=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


# Training Helpers

In [ ]:
def validate(model, args):
    criterion = nn.CrossEntropyLoss()
    total = 0
    for i, (xs, ys) in enumerate(testloader):
        images, labels = get_torch_vars(xs, ys, args.gpu)
        outputs = model(images)
        predict = torch.argmax(outputs, dim=1)
        total += torch.sum(labels == predict)
    return total.item() / (len(testloader) * BATCH_SIZE)

In [ ]:
def get_torch_vars(xs, ys, gpu=False):
    """
    Helper function to convert numpy arrays to pytorch tensors.
    If GPU is used, move the tensors to GPU.

    Args:
      xs (float numpy tenosor): greyscale input
      ys (int numpy tenosor): categorical labels
      gpu (bool): whether to move pytorch tensor to GPU
    Returns:
      Variable(xs), Variable(ys)
    """
    if gpu:
        xs = xs.cuda()
        ys = ys.cuda()
    return Variable(xs), Variable(ys)

In [ ]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

def train(model, args):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
    print("Beginning training ...")
    if args.gpu:
        model.cuda()
    start = time.time()
    # print(1)
    train_losses = []
    valid_accs = []
    for epoch in range(args.epochs):
        # Train the Model
        model.train()  # Change model to 'train' mode
        losses = []
        # print(1)
        for i, (xs, ys) in enumerate(trainloader):
            images, labels = get_torch_vars(xs, ys, args.gpu)
            # Forward + Backward + Optimize
            # print(labels)
            # print(images.size())
            optimizer.zero_grad()
            outputs = model(images)

            # print(outputs.view(BATCH_SIZE, NUM_CLASSES))

            loss = criterion(outputs.view(BATCH_SIZE, NUM_CLASSES), labels)
            print("i:", i, "loss:", loss)
            loss.backward()
            optimizer.step()
            losses.append(loss.data.item())
        avg_loss = np.mean(losses)
        train_losses.append(avg_loss)
        time_elapsed = time.time() - start
        print(
            "Epoch [%d/%d], Loss: %.4f, Time (s): %d"
            % (epoch + 1, args.epochs, avg_loss, time_elapsed)
        )
        model.eval()
        val_acc = validate(model, args)
        valid_accs.append(val_acc)
        print(
            "Epoch [%d/%d], Validation Accuracy: %.4f"
            % (epoch + 1, args.epochs, val_acc)
        )
    return model, train_losses, valid_accs

# VGG

In [ ]:
model = torch.hub.load('pytorch/vision:v0.8.1', 'vgg16', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.8.1


In [ ]:
model.cuda()
summary(model, (3, 224, 224))

In [ ]:
model.classifier[6] = nn.Linear(4096, NUM_CLASSES)

In [ ]:
args = AttrDict()
torch.cuda.empty_cache()
args_dict = {
    "gpu": True,
    "epochs": 5,
    "learning_rate": 1e-4
}
args.update(args_dict)
final_model, train_losses, valid_accs = train(model, args)

Beginning training ...
i: 0 loss: tensor(0.1979, device='cuda:0', grad_fn=<NllLossBackward>)
i: 1 loss: tensor(0.2267, device='cuda:0', grad_fn=<NllLossBackward>)
i: 2 loss: tensor(0.1920, device='cuda:0', grad_fn=<NllLossBackward>)
i: 3 loss: tensor(0.1670, device='cuda:0', grad_fn=<NllLossBackward>)
i: 4 loss: tensor(0.1903, device='cuda:0', grad_fn=<NllLossBackward>)
i: 5 loss: tensor(0.1648, device='cuda:0', grad_fn=<NllLossBackward>)
i: 6 loss: tensor(0.1236, device='cuda:0', grad_fn=<NllLossBackward>)
i: 7 loss: tensor(0.2486, device='cuda:0', grad_fn=<NllLossBackward>)
i: 8 loss: tensor(0.1594, device='cuda:0', grad_fn=<NllLossBackward>)
i: 9 loss: tensor(0.2433, device='cuda:0', grad_fn=<NllLossBackward>)
i: 10 loss: tensor(0.2053, device='cuda:0', grad_fn=<NllLossBackward>)
i: 11 loss: tensor(0.1727, device='cuda:0', grad_fn=<NllLossBackward>)
i: 12 loss: tensor(0.2534, device='cuda:0', grad_fn=<NllLossBackward>)
i: 13 loss: tensor(0.2119, device='cuda:0', grad_fn=<NllLossBack

In [ ]:
valid_accs

[0.443, 0.6042, 0.6462, 0.6986, 0.7111, 0.7288, 0.7311, 0.7244, 0.7292, 0.7327]

# Spectral Pooling

## Helpers

In [ ]:
datasets.CIFAR10

In [ ]:
!pip install tensorboardX

     |████████████████████████████████| 317kB 24.4MB/s 


In [ ]:
!rm -rf MNIST

In [ ]:
class SpectralPooling2d(nn.Module):
    def __init__(self, kernel_size):
        super(SpectralPooling2d, self).__init__()

        self.kernel_size = 2

    def crop_spectrum(self, z, H, W):
        '''
            z: [bs, c, M, N, 2]
            Return: [bs, c, H, H, 2]
        '''
        M, N = z.size(-3), z.size(-2)
        return z[..., M//2-H//2:M//2+H//2, N//2-W//2:N//2+W//2, :]

    def pad_spectrum(self, z, M, N):
        '''
            z: [bs, c, H, W, 2]
            Return: [bs, c, M, N, 2]
        '''
        H, W = z.size(-3), z.size(-2)
        z_real, z_imag = z[..., 0], z[..., 1]
        pad = torch.nn.ZeroPad2d((N-W)//2, (N-W)//2, (M-H)//2, (M-H)//2)
        return torch.stack([pad(z_real), pad(z_imag)], dim = -1)

    def treat_corner_cases(self, freq_map):
        '''
            freq_map: [bs, c, M, N, 2]
        '''
        S = [(0, 0)]
        M, N = freq_map.size(-3), freq_map.size(-2)

        if M % 2 == 1:
            S.append((M // 2, 0))
        if N % 2 == 1:
            S.append((0, N // 2))
        if M % 2 == 1 and N % 2 == 1:
            S.append((M // 2, N // 2))

        for h, w in S:
            freq_map[..., h, w, 1].zero_()

        return freq_map, S

    def remove_redundancy(self, y):
        '''
            y: input gradient map [bs, c, M, N, 2]
        '''
        z, S = self.treat_corner_cases(y)
        I = []
        M, N = y.size(-3), y.size(-2)

        for m in range(M):
            for n in range(N // 2 + 1):
                if (m, n) not in S:
                    if (m, n) not in I:
                        z[..., m, n, :].mul_(2)
                        I.append((m, n))
                        I.append(((M - m) % M, (N - n) % N))
                    else:
                        z[..., m, n, :].zero_()
        
        return z

    def recover_map(self, y):
        z, S = self.treat_corner_cases(y)
        I = []
        M, N = y.size(-3), y.size(-2)

        for m in range(M):
            for n in range(N // 2 + 1):
                if (m, n) not in S:
                    if (m, n) not in I:
                        z[..., m, n, :].mul_(0.5)
                        z[..., (M-m)%M, (N-n)%N] = z[..., m, n, :]
                        I.append((m, n))
                        I.append(((M - m) % M, (N - n) % N))
                    else:
                        z[..., m, n, :].zero_()

        return z

    def forward(self, x):
        M, N = x.size(-2), x.size(-1)
        H, W = M // self.kernel_size, N // self.kernel_size

        x_fft = torch.rfft(x, 2, onesided = False)
        crop_x_fft = self.crop_spectrum(x_fft, H, W)
        crop_x_fft, _ = self.treat_corner_cases(crop_x_fft)
        pool_x = torch.irfft(crop_x_fft, 2, onesided = False)
        return pool_x

    def backward(self, gRgx):
        H, W = gRgx.size(-2), gRgx.size(-1)
        M, N = H * self.kernel_size, W * self.kernel_size

        z = torch.rfft(gRgx, 2, onesided = False)
        z = self.remove_redundancy(z)
        z = self.pad_spectrum(z, M, N)
        z = self.recover_map(z)
        gRx = torch.irfft(z, 2, onesided = False)

        return gRx

In [ ]:
class CNNMNIST(nn.Module):
    def __init__(self):
        super(CNNMNIST, self).__init__()

        self.net1 = nn.Sequential(
            nn.Conv2d(3, 32, 3),
            nn.ReLU(True),
            Denoise(0.05),
            nn.Conv2d(32, 64, 3),
            nn.MaxPool2d(2))
        self.net2 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(14*14*64, 128),
            nn.ReLU(True),
            nn.Linear(128, 10),
            nn.LogSoftmax(1)
        )
        self.name = 'VanillaCNN'

    def forward(self, x):

        x = self.net1(x)
        # print(x.size())
        return self.net2(x)


class FCNNMNIST(nn.Module):
    def __init__(self):
        super(FCNNMNIST, self).__init__()

        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3),
            nn.ReLU(True),
            nn.Conv2d(32, 64, 3),
            GaussianPooling2d(64, 2, 2),
            nn.Flatten(),
            nn.Linear(14*14*64, 128),
            nn.ReLU(True),
            nn.Linear(128, 10),
            nn.LogSoftmax(1)
        )
        self.name = 'FourierCNN'

    def forward(self, x):
        return self.net(x)


def weight_init(net):
    for module in net.modules():
        if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
            torch.nn.init.uniform_(module.weight, -0.1, 0.1)
            if module.bias is not None:
                torch.nn.init.constant_(module.bias, 0)

In [ ]:
valid_accs

[0.4918391719745223,
 0.5516520700636943,
 0.5821058917197452,
 0.5957404458598726,
 0.617734872611465,
 0.630672770700637,
 0.6374402866242038,
 0.6448049363057324,
 0.6560509554140127,
 0.6607285031847133,
 0.662718949044586,
 0.6610270700636943,
 0.667296974522293,
 0.669187898089172,
 0.673765923566879,
 0.6716759554140127,
 0.6687898089171974,
 0.6713773885350318,
 0.6678941082802548,
 0.666202229299363]

In [ ]:
valid_accs

[0.4945262738853503,
 0.5306528662420382,
 0.5673765923566879,
 0.5818073248407644,
 0.6013136942675159,
 0.6098726114649682,
 0.6173367834394905,
 0.6291799363057324,
 0.6409235668789809,
 0.6445063694267515,
 0.6440087579617835,
 0.6463972929936306,
 0.6491839171974523,
 0.652468152866242,
 0.6555533439490446,
 0.6542595541401274,
 0.6528662420382165,
 0.6541600318471338,
 0.6549562101910829,
 0.6546576433121019]

## Train

In [ ]:
args = AttrDict()
torch.cuda.empty_cache()
model = FCNNMNIST()
model.cuda()
args_dict = {
    "gpu": True,
    "epochs": 20,
    "learning_rate": 1e-4
}
weight_init(model)
args.update(args_dict)
final_model, train_losses, valid_accs = train(model, args)

In [ ]:
spectral_pool.apply(x, 5, 5).size()

torch.Size([4, 3, 5, 5])

In [ ]:
spectral_model = torch.hub.load('pytorch/vision:v0.8.1', 'vgg16', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.8.1


In [ ]:
spectral_model.features[4] =  SpectralPooling2d(0.5).cuda()
spectral_model.features[9] = SpectralPooling2d(0.5).cuda()
spectral_model.features[16] = SpectralPooling2d(0.5).cuda()
spectral_model.features[23] = SpectralPooling2d(0.5).cuda()
# spectral_model.features[30] = SpectralPooling2d(0.5).cuda()
# spectral_model.avgpool = nn.AdaptiveAvgPool2d(output_size=(7,7)).cuda()
# spectral_model.classifier[0] = nn.Linear(18432, 4096).cuda()
spectral_model.classifier[6] = nn.Linear(4096, NUM_CLASSES).cuda()

In [ ]:
args = AttrDict()
torch.cuda.empty_cache()
spectral_model.cuda()
args_dict = {
    "gpu": True,
    "epochs": 5,
    "learning_rate": 1e-4
}
args.update(args_dict)
final_model, train_losses, valid_accs = train(spectral_model, args)

Beginning training ...
i: 0 loss: tensor(0.0974, device='cuda:0', grad_fn=<NllLossBackward>)
i: 1 loss: tensor(0.1668, device='cuda:0', grad_fn=<NllLossBackward>)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'


i: 2 loss: tensor(0.2866, device='cuda:0', grad_fn=<NllLossBackward>)
i: 3 loss: tensor(0.0660, device='cuda:0', grad_fn=<NllLossBackward>)
i: 4 loss: tensor(0.2095, device='cuda:0', grad_fn=<NllLossBackward>)
i: 5 loss: tensor(0.1667, device='cuda:0', grad_fn=<NllLossBackward>)
i: 6 loss: tensor(0.1520, device='cuda:0', grad_fn=<NllLossBackward>)
i: 7 loss: tensor(0.3332, device='cuda:0', grad_fn=<NllLossBackward>)
i: 8 loss: tensor(0.1999, device='cuda:0', grad_fn=<NllLossBackward>)
i: 9 loss: tensor(0.1747, device='cuda:0', grad_fn=<NllLossBackward>)
i: 10 loss: tensor(0.1322, device='cuda:0', grad_fn=<NllLossBackward>)
i: 11 loss: tensor(0.2256, device='cuda:0', grad_fn=<NllLossBackward>)
i: 12 loss: tensor(0.0945, device='cuda:0', grad_fn=<NllLossBackward>)
i: 13 loss: tensor(0.1739, device='cuda:0', grad_fn=<NllLossBackward>)
i: 14 loss: tensor(0.1943, device='cuda:0', grad_fn=<NllLossBackward>)
i: 15 loss: tensor(0.1424, device='cuda:0', grad_fn=<NllLossBackward>)
i: 16 loss: te

In [ ]:
valid_accs1 += valid_accs

In [ ]:
valid_accs1

[0.4823,
 0.5987,
 0.65,
 0.6883,
 0.7161,
 0.7222,
 0.7203,
 0.733,
 0.7218,
 0.7371,
 0.7344,
 0.7183,
 0.7372,
 0.7325,
 0.7327]

# HSP

In [ ]:
def _spectral_crop(input, oheight, owidth):
	cutoff_freq_h = math.ceil(oheight / 2)
	cutoff_freq_w = math.ceil(owidth / 2)

	if oheight % 2 == 1:
		if owidth % 2 == 1:
			top_left = input[:, :, :cutoff_freq_h, :cutoff_freq_w]
			top_right = input[:, :, :cutoff_freq_h, -(cutoff_freq_w-1):]
			bottom_left = input[:, :, -(cutoff_freq_h-1):, :cutoff_freq_w]
			bottom_right = input[:, :, -(cutoff_freq_h-1):, -(cutoff_freq_w-1):]
		else:
			top_left = input[:, :, :cutoff_freq_h, :cutoff_freq_w]
			top_right = input[:, :, :cutoff_freq_h, -cutoff_freq_w:]
			bottom_left = input[:, :, -(cutoff_freq_h-1):, :cutoff_freq_w]
			bottom_right = input[:, :, -(cutoff_freq_h-1):, -cutoff_freq_w:]
	else:
		if owidth % 2 == 1:
			top_left = input[:, :, :cutoff_freq_h, :cutoff_freq_w]
			top_right = input[:, :, :cutoff_freq_h, -(cutoff_freq_w-1):]
			bottom_left = input[:, :, -cutoff_freq_h:, :cutoff_freq_w]
			bottom_right = input[:, :, -cutoff_freq_h:, -(cutoff_freq_w-1):]
		else:
			top_left = input[:, :, :cutoff_freq_h, :cutoff_freq_w]
			top_right = input[:, :, :cutoff_freq_h, -cutoff_freq_w:]
			bottom_left = input[:, :, -cutoff_freq_h:, :cutoff_freq_w]
			bottom_right = input[:, :, -cutoff_freq_h:, -cutoff_freq_w:]

	top_combined = torch.cat((top_left, top_right), dim=-1)
	bottom_combined = torch.cat((bottom_left, bottom_right), dim=-1)
	all_together = torch.cat((top_combined, bottom_combined), dim=-2)

	return all_together

def _spectral_pad(input, output, oheight, owidth):
	cutoff_freq_h = math.ceil(oheight / 2)
	cutoff_freq_w = math.ceil(owidth / 2)
 
	pad = torch.zeros_like(input)

	if oheight % 2 == 1:
		if owidth % 2 == 1:
			pad[:, :, :cutoff_freq_h, :cutoff_freq_w] = output[:, :, :cutoff_freq_h, :cutoff_freq_w]
			pad[:, :, :cutoff_freq_h, -(cutoff_freq_w-1):] = output[:, :, :cutoff_freq_h, -(cutoff_freq_w-1):]
			pad[:, :, -(cutoff_freq_h-1):, :cutoff_freq_w] = output[:, :, -(cutoff_freq_h-1):, :cutoff_freq_w]
			pad[:, :, -(cutoff_freq_h-1):, -(cutoff_freq_w-1):] = output[:, :, -(cutoff_freq_h-1):, -(cutoff_freq_w-1):]
		else:
			pad[:, :, :cutoff_freq_h, :cutoff_freq_w] = output[:, :, :cutoff_freq_h, :cutoff_freq_w]
			pad[:, :, :cutoff_freq_h, -cutoff_freq_w:] = output[:, :, :cutoff_freq_h, -cutoff_freq_w:]
			pad[:, :, -(cutoff_freq_h-1):, :cutoff_freq_w] = output[:, :, -(cutoff_freq_h-1):, :cutoff_freq_w]
			pad[:, :, -(cutoff_freq_h-1):, -cutoff_freq_w:] = output[:, :, -(cutoff_freq_h-1):, -cutoff_freq_w:]
	else:
		if owidth % 2 == 1:
			pad[:, :, :cutoff_freq_h, :cutoff_freq_w] = output[:, :, :cutoff_freq_h, :cutoff_freq_w]
			pad[:, :, :cutoff_freq_h, -(cutoff_freq_w-1):] = output[:, :, :cutoff_freq_h, -(cutoff_freq_w-1):]
			pad[:, :, -cutoff_freq_h:, :cutoff_freq_w] = output[:, :, -cutoff_freq_h:, :cutoff_freq_w]
			pad[:, :, -cutoff_freq_h:, -(cutoff_freq_w-1):] = output[:, :, -cutoff_freq_h:, -(cutoff_freq_w-1):]
		else:
			pad[:, :, :cutoff_freq_h, :cutoff_freq_w] = output[:, :, :cutoff_freq_h, :cutoff_freq_w]
			pad[:, :, :cutoff_freq_h, -cutoff_freq_w:] = output[:, :, :cutoff_freq_h, -cutoff_freq_w:]
			pad[:, :, -cutoff_freq_h:, :cutoff_freq_w] = output[:, :, -cutoff_freq_h:, :cutoff_freq_w]
			pad[:, :, -cutoff_freq_h:, -cutoff_freq_w:] = output[:, :, -cutoff_freq_h:, -cutoff_freq_w:]	

	return pad	

def DiscreteHartleyTransform(input):
	fft = torch.rfft(input, 2, normalized=True, onesided=False)
	dht = fft[:, :, :, :, -2] - fft[:, :, :, :, -1]
	return dht

class SpectralPoolingFunction(Function):
	@staticmethod
	def forward(ctx, input, oheight, owidth):
		ctx.oh = oheight
		ctx.ow = owidth
		ctx.save_for_backward(input)

		# Hartley transform by RFFT
		dht = DiscreteHartleyTransform(input)

		# frequency cropping
		all_together = _spectral_crop(dht, oheight, owidth)
		# inverse Hartley transform
		dht = DiscreteHartleyTransform(all_together)
		return dht

	@staticmethod
	def backward(ctx, grad_output):
		input, = ctx.saved_variables

		# Hartley transform by RFFT
		dht = DiscreteHartleyTransform(grad_output)
		# frequency padding
		grad_input = _spectral_pad(input, dht, ctx.oh, ctx.ow)
		# inverse Hartley transform
		grad_input = DiscreteHartleyTransform(grad_input)
		return grad_input, None, None

class SpectralPool2d(nn.Module):
	def __init__(self, scale_factor):
		super(SpectralPool2d, self).__init__()
		self.scale_factor = _pair(scale_factor)
	def forward(self, input):
		H, W = input.size(-2), input.size(-1)
		h, w = math.ceil(H*self.scale_factor[0]), math.ceil(W*self.scale_factor[1])
		return SpectralPoolingFunction.apply(input, h, w)

In [ ]:
spectral_model = torch.hub.load('pytorch/vision:v0.9.0', 'vgg11')

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


In [ ]:
spectral_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): SpectralPool2d()
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): SpectralPool2d()
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): SpectralPool2d()
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): SpectralPool2d()
    (16): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace=True)
    (18): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (19): ReLU(inplace=True)
    (20): SpectralP

In [ ]:
summary(spectral_model.cuda(), (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
    SpectralPool2d-3         [-1, 64, 112, 112]               0
            Conv2d-4        [-1, 128, 112, 112]          73,856
              ReLU-5        [-1, 128, 112, 112]               0
    SpectralPool2d-6          [-1, 128, 56, 56]               0
            Conv2d-7          [-1, 256, 56, 56]         295,168
              ReLU-8          [-1, 256, 56, 56]               0
            Conv2d-9          [-1, 256, 56, 56]         590,080
             ReLU-10          [-1, 256, 56, 56]               0
   SpectralPool2d-11          [-1, 256, 28, 28]               0
           Conv2d-12          [-1, 512, 28, 28]       1,180,160
             ReLU-13          [-1, 512, 28, 28]               0
           Conv2d-14          [-1, 512,

In [ ]:
spectral_model.features[2] = SpectralPool2d(0.5).cuda()
spectral_model.features[5] = SpectralPool2d(0.5).cuda()
spectral_model.features[10] = SpectralPool2d(0.5).cuda()
spectral_model.features[15] = SpectralPool2d(0.5).cuda()
spectral_model.features[20] = SpectralPool2d(0.5).cuda()
# spectral_model.avgpool = nn.AdaptiveAvgPool2d(output_size=(6,6)).cuda()
# spectral_model.classifier[0] = nn.Linear(-1, 4096).cuda()
spectral_model.classifier[6] = nn.Linear(4096, NUM_CLASSES).cuda()

In [ ]:
args = AttrDict()
torch.cuda.empty_cache()
# spectral_model.cuda()
args_dict = {
    "gpu": True,
    "epochs": 10,
    "learning_rate": 1e-5
}
args.update(args_dict)
final_model, train_losses, valid_accs = train(spectral_model, args)

Beginning training ...
i: 0 loss: tensor(4.2541, device='cuda:0', grad_fn=<NllLossBackward>)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'


i: 1 loss: tensor(4.4168, device='cuda:0', grad_fn=<NllLossBackward>)
i: 2 loss: tensor(4.2240, device='cuda:0', grad_fn=<NllLossBackward>)
i: 3 loss: tensor(4.1100, device='cuda:0', grad_fn=<NllLossBackward>)
i: 4 loss: tensor(4.0775, device='cuda:0', grad_fn=<NllLossBackward>)
i: 5 loss: tensor(4.4661, device='cuda:0', grad_fn=<NllLossBackward>)
i: 6 loss: tensor(4.3816, device='cuda:0', grad_fn=<NllLossBackward>)
i: 7 loss: tensor(4.1534, device='cuda:0', grad_fn=<NllLossBackward>)
i: 8 loss: tensor(4.3883, device='cuda:0', grad_fn=<NllLossBackward>)
i: 9 loss: tensor(4.2119, device='cuda:0', grad_fn=<NllLossBackward>)
i: 10 loss: tensor(4.1986, device='cuda:0', grad_fn=<NllLossBackward>)
i: 11 loss: tensor(4.1837, device='cuda:0', grad_fn=<NllLossBackward>)
i: 12 loss: tensor(4.0979, device='cuda:0', grad_fn=<NllLossBackward>)
i: 13 loss: tensor(4.2578, device='cuda:0', grad_fn=<NllLossBackward>)
i: 14 loss: tensor(4.2489, device='cuda:0', grad_fn=<NllLossBackward>)
i: 15 loss: ten

KeyboardInterrupt: ignored

# Gaussian Pool

In [ ]:
class GaussianPooling2d(nn.AvgPool2d):
    def __init__(self, num_features, kernel_size, stride=None, padding=0, ceil_mode=False,
                 count_include_pad=True, hidden_node=None, stochasticity='HWCN', eps=1e-6):
        if stochasticity != 'HWCN' and stochasticity != 'CN' and stochasticity is not None:
            raise ValueError("gaussian pooling stochasticity has to be 'HWCN'/'CN' or None, "
                         "but got {}".format(stochasticity))
        if hidden_node is None:
            hidden_node = num_features // 2

        super(GaussianPooling2d, self).__init__(kernel_size, stride=stride, padding=padding, ceil_mode=ceil_mode,
                    count_include_pad=count_include_pad)
        self.eps = eps
        self.stochasticity = stochasticity

        self.ToHidden = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Conv2d(num_features, hidden_node, kernel_size=1,  padding=0, bias=True),
            nn.BatchNorm2d(hidden_node),
            nn.ReLU(False),
        )
        self.ToMean = nn.Sequential(
            nn.Conv2d(hidden_node, num_features, kernel_size=1,  padding=0, bias=True),
            nn.BatchNorm2d(num_features),
        )
        self.ToSigma = nn.Sequential(
            nn.Conv2d(hidden_node, num_features, kernel_size=1,  padding=0, bias=True),
            nn.BatchNorm2d(num_features),
            nn.Sigmoid()
        )
        self.activation = nn.Softplus()
        
    def forward(self, input):
        mu0 = F.avg_pool2d(input, self.kernel_size, self.stride, self.padding, self.ceil_mode, self.count_include_pad)
        sig0= F.avg_pool2d(input**2, self.kernel_size, self.stride, self.padding, self.ceil_mode, self.count_include_pad)
        sig0= torch.sqrt(torch.clamp(sig0 - mu0**2, self.eps))

        Z = self.ToHidden(input)
        MU = self.ToMean(Z)

        if self.training and self.stochasticity is not None:
            SIGMA = self.ToSigma(Z)
            if self.stochasticity == 'HWCN':
                size = sig0.size()
            else:
                size = [sig0.size(0), sig0.size(1), 1, 1]
            W = self.activation(MU + SIGMA * 
                torch.randn(size, dtype=sig0.dtype, layout=sig0.layout, device=sig0.device))
        else:
            W = self.activation(MU)

        return mu0 + W*sig0

In [ ]:
gaussian_model = torch.hub.load('pytorch/vision:v0.8.1', 'vgg16', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.8.1


In [ ]:
gaussian_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): GaussianPooling2d(
      kernel_size=2, stride=2, padding=0
      (ToHidden): Sequential(
        (0): AdaptiveAvgPool2d(output_size=(1, 1))
        (1): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
        (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): ReLU()
      )
      (ToMean): Sequential(
        (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (ToSigma): Sequential(
        (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Sigmoid()
      )
      (activation): So

In [ ]:
gaussian_model.features[4] =  GaussianPooling2d(64,2,2).cuda()
gaussian_model.features[9] = GaussianPooling2d(128,2,2).cuda()
gaussian_model.features[16] = GaussianPooling2d(256,2,2).cuda()
gaussian_model.features[23] = GaussianPooling2d(512,2,2).cuda()
gaussian_model.features[30] = GaussianPooling2d(512,2,2).cuda()
# spectral_model.avgpool = nn.AdaptiveAvgPool2d(output_size=(6,6)).cuda()
# spectral_model.classifier[0] = nn.Linear(18432, 4096).cuda()
gaussian_model.classifier[6] = nn.Linear(4096, NUM_CLASSES).cuda()

In [ ]:
args = AttrDict()
torch.cuda.empty_cache()
args_dict = {
    "gpu": True,
    "epochs": 5,
    "learning_rate": 1e-4
}
args.update(args_dict)
final_model, train_losses, valid_accs = train(gaussian_model, args)

Beginning training ...
i: 0 loss: tensor(0.2673, device='cuda:0', grad_fn=<NllLossBackward>)
i: 1 loss: tensor(0.1494, device='cuda:0', grad_fn=<NllLossBackward>)
i: 2 loss: tensor(0.2568, device='cuda:0', grad_fn=<NllLossBackward>)
i: 3 loss: tensor(0.2381, device='cuda:0', grad_fn=<NllLossBackward>)
i: 4 loss: tensor(0.2311, device='cuda:0', grad_fn=<NllLossBackward>)
i: 5 loss: tensor(0.2163, device='cuda:0', grad_fn=<NllLossBackward>)
i: 6 loss: tensor(0.4631, device='cuda:0', grad_fn=<NllLossBackward>)
i: 7 loss: tensor(0.1935, device='cuda:0', grad_fn=<NllLossBackward>)
i: 8 loss: tensor(0.2188, device='cuda:0', grad_fn=<NllLossBackward>)
i: 9 loss: tensor(0.2347, device='cuda:0', grad_fn=<NllLossBackward>)
i: 10 loss: tensor(0.1845, device='cuda:0', grad_fn=<NllLossBackward>)
i: 11 loss: tensor(0.3737, device='cuda:0', grad_fn=<NllLossBackward>)
i: 12 loss: tensor(0.1530, device='cuda:0', grad_fn=<NllLossBackward>)
i: 13 loss: tensor(0.3325, device='cuda:0', grad_fn=<NllLossBack

In [ ]:
valid_accs

[0.4404,
 0.5876,
 0.6317,
 0.6751,
 0.6949,
 0.7183,
 0.7359,
 0.7526,
 0.7671,
 0.7341]